In [ ]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from time import time 
import datetime
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
import time

In [ ]:
data1 = pd.read_excel(io = 'F:\\feature.xlsx',header=None)
data2 = pd.read_excel(io = 'F:\\cfeature.xlsx',header=None)

In [ ]:
label1 = pd.read_excel(io = 'F:\\label2.xlsx',header=None)
label1=label1.values.ravel()   

label2 = pd.read_excel(io = 'F:\\clabel2.xlsx',header=None)
label2=label2.values.ravel()   

In [ ]:
traindata = data1
testdata = data2

In [ ]:
data = pd.concat([traindata,testdata], axis=0)

In [ ]:
scaler = StandardScaler().fit(data)
data= scaler.transform(data)
data = pd.DataFrame(data)

In [ ]:
Xtrain = data.iloc[0:288]
Xtest = data.iloc[288:576]

Ytrain = label1
Ytest = label2

In [ ]:
X = Xtrain
y = Ytrain

In [ ]:
print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier


# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=9
                              ,random_state=66
                             )

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
clf = ExtraTreesClassifier(n_estimators=9
                          ,random_state=66
                          )
clf = clf.fit(X, y)

In [ ]:
score1 = []
score2 = []
rrange = np.arange(20,50) 
for i in rrange:
    model = SelectFromModel(clf
                            , prefit=True
                            ,threshold=-np.inf
                           ,max_features =i)
    X_new = model.transform(X)
    
    gamma_range = np.logspace(-10,1,20) 
#     coef0_range = np.linspace(0,5,30)
#     C_range = np.logspace(-2, 2, 10)
    C_range = np.linspace(0.001,30,50) 

    param_grid = dict(gamma = gamma_range
    #                   ,coef0 = coef0_range
                      ,C=C_range
                     )

    cv = StratifiedShuffleSplit(n_splits=10
                                , test_size=0.3
                                , random_state=123
                               ) 
    grid = GridSearchCV(SVC(kernel = "rbf",cache_size=8000), 
    param_grid=param_grid, cv=cv
                       ,n_jobs=3) 
    grid.fit(X_new, y)

    print("The best parameters are %s with a score of %0.5f" % (grid.best_params_, 
    grid.best_score_)) 
    # print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))

    score1.append(grid.best_score_)
    score2.append(grid.best_params_)
    
print(" acc          {}\n".format(max(score1))
     ,"r              {}\n".format(rrange[score1.index(max(score1))])
    ,"Candr             {}\n".format(score2[score1.index(max(score1))])
     )
plt.plot(rrange,score1) 
plt.show()    

In [ ]:
ab = indices[0:rrange[score1.index(max(score1))]]
canshu = score2[score1.index(max(score1))]
XtrainS = Xtrain.iloc[:,ab]
XtestS = Xtest.iloc[:,ab]
Ytrain = label1
Ytest = label2

In [ ]:
clf = SVC(kernel="rbf",C = canshu['C'],gamma = canshu['gamma'],cache_size=7000).fit(XtrainS,Ytrain)
clf.score(XtestS,Ytest)